In [19]:
import pandas as pd
import geocoder

In [20]:
data = pd.read_csv('18k_Projects.csv')

In [21]:
data.columns

Index(['Id', 'Name', 'Url', 'State', 'Currency', 'Top Category', 'Category',
       'Creator', 'Location', 'Updates', 'Comments', 'Rewards', 'Goal',
       'Pledged', 'Backers', 'Start', 'End', 'Duration in Days',
       'Facebook Connected', 'Facebook Friends', 'Facebook Shares',
       'Has Video', 'Latitude', 'Longitude', 'Start Timestamp (UTC)',
       'End Timestamp (UTC)', 'Creator Bio', 'Creator Website',
       'Creator - # Projects Created', 'Creator - # Projects Backed',
       '# Videos', '# Images', '# Words (Description)',
       '# Words (Risks and Challenges)', '# FAQs'],
      dtype='object')

In [22]:
f_data = data[['Id', 'Name', 'State', 'Top Category', 'Creator', 'Location', 'Goal', 'Pledged', 'Backers', 'Duration in Days', 'Facebook Connected', 'Facebook Friends', 'Facebook Shares', 'Creator - # Projects Created', 'Creator - # Projects Backed', 'Latitude', 'Longitude']]

In [80]:
!pip install reverse_geocoder

  Stored in directory: C:\Users\shinu\AppData\Local\pip\Cache\wheels\47\05\50\b1350ff094ef91e082665b4a2f9ca551f8acea4aa55d796b26
Successfully built reverse-geocoder


In [28]:
f_data.groupby(['Location', 'Top Category'])['State'].sum()['Portland'].sort_values()

Top Category
Journalism       1.0
Crafts           2.0
Photography      2.0
Fashion          3.0
Theater          4.0
Comics           9.0
Design           9.0
Games            9.0
Technology       9.0
Art             14.0
Publishing      14.0
Film & Video    18.0
Food            22.0
Music           35.0
Name: State, dtype: float64

In [90]:
f_data.to_csv('hetsvi_cleaned.csv')

In [5]:
f_data = f_data.dropna(subset=['Latitude','Longitude'])

In [106]:
coordinates = tuple(list(f_data.apply(lambda x: (x['Latitude'], x['Longitude']), axis=1)))

In [107]:
results = rg.search(coordinates)

In [108]:
in_us = []
region = []
for i in results:
    in_us.append(i['cc']=='US')
    region.append(i['admin1'])

In [109]:
f_data['IN US'] = in_us
f_data['region'] = region

In [110]:
f_data = f_data.loc[f_data['IN US']==True]

In [111]:
f_data = f_data.drop('IN US',axis=1)

In [112]:
f_data.shape

(7543, 18)

In [26]:
f_data['State']=f_data['State'].replace({'failed':False,'successful':True})

C:\Users\shinu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [168]:
final_maps= f_data.groupby('region')['State'].sum().append(pd.Series({'District of Columbia':0}))

In [138]:
f_data['Duration in Days'].unique()

array([30, 33, 60, 40, 36, 31, 45, 28, 26,  9, 46, 14, 20, 42, 59, 47, 35,
       32, 50, 34, 25, 41, 57, 53, 39, 15, 29, 22, 10, 24, 21, 55, 27, 18,
       16, 19, 23, 44,  6, 49, 51, 37, 12, 43, 56, 48, 11,  7, 38, 17, 54,
       58,  8, 13, 52,  3,  5,  1,  4], dtype=int64)

In [152]:
pop_data = pd.read_csv('nst-est2018-alldata.csv')

In [163]:
pop_vals = pop_data.loc[[x in list(final_maps.index) for x in pop_data['NAME']]][['NAME', 'POPESTIMATE2014']].set_index('NAME')

In [183]:
standard_pop = {}
for i in pop_vals.index:
    standard_pop[i] = (final_maps[i]/pop_vals.loc[i]['POPESTIMATE2014'])*1000000

In [188]:
pd.Series(standard_pop).sort_values(ascending=False).to_csv('standard_final_map.csv')

C:\Users\shinu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
